In [1]:
pip install spacy transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [3]:
#sample data

texts = [
    "order two pizzas",
    "Track my delivery",
    "Add garlic bread to my order",
    "What promotions are available",
    "cancel my order"
]
intents = [
    "order_food",
    "track_order",
    "update_order",
    "ask_promotions",
    "cancel_order"
]

entities = [
    {"type": "food", "text": "pizzas", "start": 6, "end": 12},
    {"type": "order_action", "text": "track", "start": 0, "end": 5},
    {"type": "food", "text": "garlic bread", "start": 4, "end": 16},
    {"type": "inquiry", "text": "promotions", "start": 5, "end": 16},
    {"type": "order_action", "text": "cancel", "start": 0, "end": 6}
]

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

In [5]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels = len(set(intents)))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#example of classifying an intent
def classify_intent(text):
    inputs = tokenizer(text, return_tensors ="pt")
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=-1)
    return intents[torch.argmax(probs)]

In [7]:
#test the function
print(classify_intent("how can i track  my order?"))

track_order


In [8]:
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example

In [9]:
#load a blank spacy model
nlp = spacy.blank("en")

#create  the NER component if it doesn't exist
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last = True)

#add enetity labels to the NER
for entity in entities:
    ner.add_label(entity["type"])



In [10]:
#preparing training data
doc_bin = DocBin()
for text, entity in zip(texts, entities):
    doc = nlp.make_doc(text)
    ents = [(entity["start"], entity["end"], entity["type"])]
    example = Example.from_dict(doc, {"entities": ents})
    doc_bin.add(example.reference)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "order two pizzas" with entities "[(6, 12, 'food')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "What promotions are available" with entities "[(5, 16, 'inquiry')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [11]:
#train the NER model
nlp.initialize()
for i in range(10):
    losses = {}
    nlp.update([example], losses =losses)
    print(f"Loss: {losses}")


Loss: {'ner': 2.785714030265808}
Loss: {'ner': 2.7224084734916687}
Loss: {'ner': 2.5853646993637085}
Loss: {'ner': 2.3696327209472656}
Loss: {'ner': 2.027519941329956}
Loss: {'ner': 1.5068694949150085}
Loss: {'ner': 0.9606377631425858}
Loss: {'ner': 0.7513259649276733}
Loss: {'ner': 0.4275237172842026}
Loss: {'ner': 0.21900997124612331}


In [14]:
#test the NER model
test_doc = nlp("i would like to order a  pizza and a coke")
print([(ent.text, ent.label_) for ent in test_doc.ents])

[]


In [15]:
def handle_query(text):
    intent = classify_intent(text)
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return {"intent": intent, "entities" : entities}



In [16]:
response = handle_query("Can i get a discount?")
print(response)

{'intent': 'track_order', 'entities': []}
